In [1]:
!pip install -Uq requests pandas python-dotenv numpy openai tenacity matplotlib plotly scipy scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [21]:
df = pd.read_csv('data/startups.csv')
df = df.head(10)
df

,Id,Name,Website,Logo,Description,Founded,FoundersName,FoundersRelatedLocations,TotalRaised,Valuation,...,Phone,FacebookReview,FacebookLikes,FacebookCheckIns,MembersCount,Tags,LastActivity,PrimarySource,CreatedTimeStamp,UpdatedTimeStamp
0,2c379cf0-9a84-4706-b7ff-4a43b662bca2,ABA Cooltrans,http://aba.com.vn/,https://parsers.vc/logo/2c379cf0-9a84-4706-b7f...,ABA is the leading service provider in the fie...,2008.0,NaN,NaN,NaN,NaN,...,+84 28 3920 8503,NaN,523.0,NaN,51-200,Business\r\nE-commerce\r\nFinTech\r\nLogistics...,8/4/2016 12:00:00 AM,FundWebsite,6/27/2021 4:41:21 PM,12/7/2022 4:12:38 PM
1,23df578c-71f3-48d5-add1-727d1b0768ff,Acuity Trading Ltd,http://acuitytrading.com/,https://parsers.vc/logo/23df578c-71f3-48d5-add...,Acuity Trading is a fintech company. We use bi...,2013.0,NaN,NaN,NaN,NaN,...,+44 20 8133 0095,NaN,245.0,NaN,NaN,Broker\r\nData\r\nE-commerce\r\nFinTech\r\nInv...,7/9/2024 12:00:00 AM,FundWebsite,8/20/2020 6:21:17 AM,7/12/2024 7:19:48 PM
2,32b84046-27a4-4c31-91e2-83e41150f5f5,Advarra,http://advarra.com/,https://parsers.vc/logo/32b84046-27a4-4c31-91e...,Advarra advances the way clinical research is ...,1983.0,NaN,NaN,NaN,NaN,...,+1 410-884-2900,NaN,23.0,NaN,501-1000,Assisted\r\nBioTech\r\nHardware\r\nHuman\r\nMa...,1/25/2024 12:00:00 AM,FundWebsite,6/16/2020 1:11:15 PM,2/19/2024 4:03:40 PM
3,7f533dbf-e9d5-4573-8d94-06c1b3360450,Alantys,http://alantys.com/,https://parsers.vc/logo/7f533dbf-e9d5-4573-8d9...,We help OEMs & EMS with their supply chain iss...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Content Distribution\r\nFast\r\nIndustry\r\nSu...,3/10/2024 7:30:13 PM,FundWebsite,3/17/2024 8:59:42 AM,8/16/2024 8:33:49 PM
4,ad20878b-1395-432f-8eb0-aae01db1d7c2,Alesi Surgical,http://alesi-surgical.com/,https://parsers.vc/logo/ad20878b-1395-432f-8eb...,"Developer of Ultravision, the revolutionary sy...",2009.0,NaN,NaN,6743967.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Development\r\nEnergyTech\r\nExchange\r\nHardw...,6/1/2017 12:00:00 AM,FundWebsite,9/17/2020 7:41:35 AM,1/11/2024 2:52:52 AM
5,ad402473-48fe-4dd5-9977-98fad1a510c4,Allegro,http://allegro.pl/,https://parsers.vc/logo/ad402473-48fe-4dd5-997...,Allegro - Najlepsze ceny oraz gwarancja bezpie...,1999.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1001-5000,B2C\r\nE-commerce\r\nMarketplace\r\nOnline\r\n...,10/11/2023 12:00:00 AM,FundWebsite,8/6/2020 1:38:53 PM,1/13/2024 1:49:21 AM
6,9771ff71-74be-44f4-bddc-9b41029110e9,Alvest,http://alvest.fr,https://parsers.vc/logo/9771ff71-74be-44f4-bdd...,Our mission is to provide the highest quality ...,2006.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1001-5000,Aerospace\r\nContent Distribution\r\nDesign\r\...,9/12/2019 12:00:00 AM,FundWebsite,10/15/2020 6:18:44 PM,6/14/2024 2:06:35 PM
7,57a6ecaf-b1ec-423f-b76f-0afd405856ca,Ample,http://amplemeal.com/,https://parsers.vc/logo/57a6ecaf-b1ec-423f-b76...,Optimally nutritious meal-in-a-bottle for busy...,2015.0,NaN,NaN,2000000.0,NaN,...,+1 800-267-5389,NaN,13272.0,NaN,11-50,Development\r\nEdTech\r\nFast\r\nFoodTech\r\nH...,3/4/2024 12:00:00 AM,FundWebsite,6/10/2020 9:53:03 AM,3/7/2024 3:16:35 PM
8,51f24321-52e7-4d01-ba7f-dbbf2374a8aa,Ampogee,http://ampogee.com/,https://parsers.vc/logo/51f24321-52e7-4d01-ba7...,Ampogee redefines how manufacturers look at pe...,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1-10,Analytics\r\nEquipment\r\nIndustry\r\nIT\r\nMa...,NaN,FundWebsite,2/25/2021 10:03:36 AM,10/26/2022 5:41:29 AM
9,d759b5fe-5640-475e-81e6-fd4e2493b69a,Apex HCM,http://apexhcm.com/,https://parsers.vc/logo/d759b5fe-5640-475e-81e...,Apex HCM is a market leader in licensing cloud...,2008.0,NaN,NaN,NaN,NaN,...,(877) 750-2739,NaN,418.0,NaN,51-200,Business\r\nHCM\r\nHR\r\nHuman\r\nMarket\r\nPr...,1/17/2023 12:00:00 AM,FundWebsite,6/16/2020 12:53:46 PM,3/6/2023 7:20:48 PM


In [22]:
# Define a function to combine columns with their names and values
def combine_columns_with_names(row):
    # Exclude the 'id' column from the combination
    return ' | '.join(f"{col}: {row[col]}" for col in df.columns if col in ['Name', 'Website', 'Description', 'Founded', 'Tags', 'PrimarySource'])

# Create a new column that concatenates all columns except 'id' with names and values
df['Text'] = df.apply(combine_columns_with_names, axis=1)

# Create a new DataFrame with only the 'Combined' column
df_combined = df[['Id','Text']]

# Display the new DataFrame
df_combined = df_combined.head()

In [2]:
import getpass

API_KEY = getpass.getpass("OpenAI API Key: ")

: 

In [23]:
# Best practice
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str) -> list[float]:
    return str(client.embeddings.create(input=[text], model='text-embedding-3-small').data[0].embedding)

df_combined['embedding'] = df_combined['Text'].apply(get_embedding)


In [24]:
df_combined["embedding"] = df_combined.embedding.apply(literal_eval).apply(np.array)
df_combined

,Id,Text,embedding
0,2c379cf0-9a84-4706-b7ff-4a43b662bca2,Name: ABA Cooltrans | Website: http://aba.com....,"[-0.022274695336818695, 0.0015652802539989352,..."
1,23df578c-71f3-48d5-add1-727d1b0768ff,Name: Acuity Trading Ltd | Website: http://acu...,"[-0.04567258059978485, 0.0021183413919061422, ..."
2,32b84046-27a4-4c31-91e2-83e41150f5f5,Name: Advarra | Website: http://advarra.com/ |...,"[-0.023613974452018738, -0.005993706174194813,..."
3,7f533dbf-e9d5-4573-8d94-06c1b3360450,Name: Alantys | Website: http://alantys.com/ |...,"[-0.034609127789735794, 0.004136163741350174, ..."
4,ad20878b-1395-432f-8eb0-aae01db1d7c2,Name: Alesi Surgical | Website: http://alesi-s...,"[-0.032135654240846634, -0.0407610684633255, -..."


In [25]:
from utils import get_embedding, cosine_similarity

# search through the reviews for a specific product
def search_reviews(df, product_description, n=3):
    product_embedding = get_embedding(
        product_description,
        model="text-embedding-3-small"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )
    return results


results = search_reviews(df_combined, "fintech companies", n=3)

results

,Id,Text,embedding,similarity
1,23df578c-71f3-48d5-add1-727d1b0768ff,Name: Acuity Trading Ltd | Website: http://acu...,"[-0.04567258059978485, 0.0021183413919061422, ...",0.454300
4,ad20878b-1395-432f-8eb0-aae01db1d7c2,Name: Alesi Surgical | Website: http://alesi-s...,"[-0.032135654240846634, -0.0407610684633255, -...",0.315257
2,32b84046-27a4-4c31-91e2-83e41150f5f5,Name: Advarra | Website: http://advarra.com/ |...,"[-0.023613974452018738, -0.005993706174194813,...",0.304592
